In [ ]:
# ===== CELL 1 : INSTALLATION =====
print("🔧 Installation des packages...\n")

!pip install -q ultralytics gradio

import torch
from ultralytics import YOLO

print("=" * 60)
print("✅ INSTALLATION TERMINÉE")
print("=" * 60)

print(f"\n📦 PyTorch : {torch.__version__}")
print(f"🎮 GPU disponible : {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🚀 GPU : {torch.cuda.get_device_name(0)}")
    print("   ⏱️ Entraînement : ~15-20 minutes")
else:
    print("⚠️ CPU seulement (pas de GPU)")
    print("   ⏱️ Entraînement : ~45-60 minutes")
    print("\n💡 Pour activer le GPU :")
    print("   Exécution > Modifier le type d'exécution > T4 GPU")

print("\n" + "=" * 60)

🔧 Installation des packages...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ INSTALLATION TERMINÉE

📦 PyTorch : 2.9.0+cu126
🎮 GPU disponible : False
⚠️ CPU seulement (pas de GPU)
   ⏱️ Entraînement : ~45-60 minutes

💡 Pour activer le GPU :
   Exécution > Modifier le type d'exécution > T4 GPU



In [ ]:
# ===== CELL 2 : UPLOAD ET ORGANISATION =====
from google.colab import files
import zipfile
import os
import shutil
from PIL import Image
import random

print("=" * 60)
print("📤 UPLOAD DES 3 FICHIERS ZIP")
print("   SDB : Bas / Moyen / Haut")
print("=" * 60)

print("\n💡 Upload tes 3 fichiers ZIP :")
print("   1. sdb_bas.zip (50 images)")
print("   2. sdb_moyen.zip (50 images)")
print("   3. sdb_haut.zip (50 images)\n")

uploaded = files.upload()

# Créer structure temporaire
os.makedirs("dataset_sdb/raw/bas", exist_ok=True)
os.makedirs("dataset_sdb/raw/moyen", exist_ok=True)
os.makedirs("dataset_sdb/raw/haut", exist_ok=True)

print("\n📦 Extraction et traitement...\n")

stats = {'bas': 0, 'moyen': 0, 'haut': 0, 'total': 0, 'rejected': 0}

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"   📂 {filename}")

        # Détection automatique de la classe
        fname_lower = filename.lower()
        if 'bas' in fname_lower or 'basic' in fname_lower or 'renov' in fname_lower:
            classe = 'bas'
        elif 'moyen' in fname_lower or 'medium' in fname_lower or 'standard' in fname_lower:
            classe = 'moyen'
        elif 'haut' in fname_lower or 'high' in fname_lower or 'neuf' in fname_lower:
            classe = 'haut'
        else:
            print(f"      ⚠️ Classe non détectée, considéré comme 'moyen'")
            classe = 'moyen'

        print(f"      → Classe détectée : {classe}")

        # Extraction
        temp = f"temp_{filename.replace('.zip', '').replace(' ', '_')}"
        with zipfile.ZipFile(filename, 'r') as z:
            z.extractall(temp)

        # Traiter les images
        count = 0
        rejected = 0

        for root, dirs, files_list in os.walk(temp):
            for file in files_list:
                if file.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
                    filepath = os.path.join(root, file)

                    try:
                        img = Image.open(filepath)
                        w, h = img.size

                        # Filtre minimal (accepter même petites images)
                        if w < 100 or h < 100:
                            rejected += 1
                            continue

                        # Ratio max
                        ratio = max(w, h) / min(w, h)
                        if ratio > 4:
                            rejected += 1
                            continue

                        # Convertir en RGB
                        if img.mode != 'RGB':
                            img = img.convert('RGB')

                        # Sauvegarder
                        dst = f"dataset_sdb/raw/{classe}/{classe}_{count:04d}.jpg"
                        img.save(dst, 'JPEG', quality=85)
                        count += 1

                    except Exception as e:
                        rejected += 1

        stats[classe] += count
        stats['total'] += count
        stats['rejected'] += rejected

        print(f"      ✅ {count} images valides")
        if rejected > 0:
            print(f"      🗑️ {rejected} images rejetées")
        print()

        # Nettoyer
        shutil.rmtree(temp, ignore_errors=True)
        os.remove(filename)

print("\n" + "=" * 60)
print("✅ IMPORT TERMINÉ")
print("=" * 60)

print(f"\n📊 STATISTIQUES :")
print(f"   🔴 Bas (travaux nécessaires) : {stats['bas']} images")
print(f"   🟡 Moyen (état correct)       : {stats['moyen']} images")
print(f"   🟢 Haut (excellent état)      : {stats['haut']} images")
print(f"   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"   ✅ TOTAL                       : {stats['total']} images")

if stats['rejected'] > 0:
    print(f"   🗑️ Rejetées                    : {stats['rejected']} images")

print("\n🎯 Répartition équilibrée : ", end="")
min_class = min(stats['bas'], stats['moyen'], stats['haut'])
max_class = max(stats['bas'], stats['moyen'], stats['haut'])
if max_class / min_class < 1.5:
    print("✅ Excellent !")
else:
    print(f"⚠️ Ratio {max_class/min_class:.1f}:1")

print("\n🚀 PROCHAINE ÉTAPE : Cell 3 (Organisation train/val/test)")
print("=" * 60)

📤 UPLOAD DES 3 FICHIERS ZIP
   SDB : Bas / Moyen / Haut

💡 Upload tes 3 fichiers ZIP :
   1. sdb_bas.zip (50 images)
   2. sdb_moyen.zip (50 images)
   3. sdb_haut.zip (50 images)



Saving sdb_bas.zip to sdb_bas.zip
Saving sdb_moyen.zip to sdb_moyen.zip
Saving sdb_haut.zip to sdb_haut.zip

📦 Extraction et traitement...

   📂 sdb_bas.zip
      → Classe détectée : bas
      ✅ 50 images valides

   📂 sdb_moyen.zip
      → Classe détectée : moyen
      ✅ 50 images valides

   📂 sdb_haut.zip
      → Classe détectée : haut
      ✅ 50 images valides


✅ IMPORT TERMINÉ

📊 STATISTIQUES :
   🔴 Bas (travaux nécessaires) : 50 images
   🟡 Moyen (état correct)       : 50 images
   🟢 Haut (excellent état)      : 50 images
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ✅ TOTAL                       : 150 images

🎯 Répartition équilibrée : ✅ Excellent !

🚀 PROCHAINE ÉTAPE : Cell 3 (Organisation train/val/test)


In [ ]:
# ===== CELL 3 : ORGANISATION TRAIN/VALID/TEST =====
import os
import shutil
import random
from pathlib import Path

print("=" * 60)
print("📁 ORGANISATION DU DATASET")
print("   Split aléatoire : 70% train / 20% valid / 10% test")
print("=" * 60)

# Créer la structure finale
for split in ['train', 'valid', 'test']:
    for classe in ['bas', 'moyen', 'haut']:
        os.makedirs(f"dataset_sdb/{split}/{classe}", exist_ok=True)

# Seed pour reproductibilité
random.seed(42)

print("\n🔀 Mélange et répartition aléatoire...\n")

for classe in ['bas', 'moyen', 'haut']:
    # Récupérer toutes les images de cette classe
    raw_path = f"dataset_sdb/raw/{classe}"
    images = [os.path.join(raw_path, f) for f in os.listdir(raw_path)
              if f.endswith('.jpg')]

    # MÉLANGE ALÉATOIRE ← IMPORTANT !
    random.shuffle(images)

    # Calculer les splits
    total = len(images)
    train_count = int(total * 0.70)
    valid_count = int(total * 0.20)

    # Répartir
    train_imgs = images[:train_count]
    valid_imgs = images[train_count:train_count + valid_count]
    test_imgs = images[train_count + valid_count:]

    # Copier dans les bons dossiers
    for img_path in train_imgs:
        shutil.copy(img_path, f"dataset_sdb/train/{classe}/{Path(img_path).name}")

    for img_path in valid_imgs:
        shutil.copy(img_path, f"dataset_sdb/valid/{classe}/{Path(img_path).name}")

    for img_path in test_imgs:
        shutil.copy(img_path, f"dataset_sdb/test/{classe}/{Path(img_path).name}")

    # Affichage
    emoji = {'bas': '🔴', 'moyen': '🟡', 'haut': '🟢'}[classe]
    print(f"   {emoji} {classe.upper()}:")
    print(f"      Train : {len(train_imgs)} images")
    print(f"      Valid : {len(valid_imgs)} images")
    print(f"      Test  : {len(test_imgs)} images")
    print(f"      Total : {total} images")
    print()

# Résumé global
total_train = sum([len(os.listdir(f"dataset_sdb/train/{c}")) for c in ['bas', 'moyen', 'haut']])
total_valid = sum([len(os.listdir(f"dataset_sdb/valid/{c}")) for c in ['bas', 'moyen', 'haut']])
total_test = sum([len(os.listdir(f"dataset_sdb/test/{c}")) for c in ['bas', 'moyen', 'haut']])

print("=" * 60)
print("✅ ORGANISATION TERMINÉE")
print("=" * 60)

print(f"\n📊 RÉSUMÉ GLOBAL :")
print(f"   📚 Train : {total_train} images ({total_train/(total_train+total_valid+total_test)*100:.0f}%)")
print(f"   ✓ Valid : {total_valid} images ({total_valid/(total_train+total_valid+total_test)*100:.0f}%)")
print(f"   🧪 Test  : {total_test} images ({total_test/(total_train+total_valid+total_test)*100:.0f}%)")
print(f"   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"   📁 TOTAL : {total_train + total_valid + total_test} images")

print("\n💡 Structure créée :")
print("""
dataset_sdb/
├── train/
│   ├── bas/     (70% des images - mélangées)
│   ├── moyen/   (70% des images - mélangées)
│   └── haut/    (70% des images - mélangées)
├── valid/
│   ├── bas/     (20% des images - mélangées)
│   ├── moyen/   (20% des images - mélangées)
│   └── haut/    (20% des images - mélangées)
└── test/
    ├── bas/     (10% des images - mélangées)
    ├── moyen/   (10% des images - mélangées)
    └── haut/    (10% des images - mélangées)
""")

print("🚀 PROCHAINE ÉTAPE : Cell 4 (Entraînement)")
print("=" * 60)

DATASET_PATH = "dataset_sdb"

📁 ORGANISATION DU DATASET
   Split aléatoire : 70% train / 20% valid / 10% test

🔀 Mélange et répartition aléatoire...

   🔴 BAS:
      Train : 35 images
      Valid : 10 images
      Test  : 5 images
      Total : 50 images

   🟡 MOYEN:
      Train : 35 images
      Valid : 10 images
      Test  : 5 images
      Total : 50 images

   🟢 HAUT:
      Train : 35 images
      Valid : 10 images
      Test  : 5 images
      Total : 50 images

✅ ORGANISATION TERMINÉE

📊 RÉSUMÉ GLOBAL :
   📚 Train : 105 images (70%)
   ✓ Valid : 30 images (20%)
   🧪 Test  : 15 images (10%)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   📁 TOTAL : 150 images

💡 Structure créée :

dataset_sdb/
├── train/
│   ├── bas/     (70% des images - mélangées)
│   ├── moyen/   (70% des images - mélangées)
│   └── haut/    (70% des images - mélangées)
├── valid/
│   ├── bas/     (20% des images - mélangées)
│   ├── moyen/   (20% des images - mélangées)
│   └── haut/    (20% des images - mélangées)
└── test/
    ├── bas/     (10% des i

In [ ]:
# ===== CELL 4 : ENTRAÎNEMENT YOLOV8 =====
from ultralytics import YOLO
import torch

print("=" * 60)
print("🚀 ENTRAÎNEMENT YOLOV8 CLASSIFICATION")
print("   Modèle : Salles de Bain (3 classes)")
print("=" * 60)

# Détecter GPU/CPU
device = 0 if torch.cuda.is_available() else 'cpu'
device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"

print(f"\n💻 Device : {device_name}")
print(f"⏱️  Durée estimée : {'15-20 min' if device == 0 else '45-60 min'}")

print("\n📋 Configuration :")
print("   Modèle     : YOLOv8n-cls (nano, rapide)")
print("   Classes    : Bas / Moyen / Haut")
print("   Epochs     : 100 (avec early stopping)")
print("   Patience   : 15 (arrêt si pas d'amélioration)")
print("   Image size : 224×224")
print("   Batch size : 16")
print("\n" + "=" * 60)
print("🔥 DÉMARRAGE DE L'ENTRAÎNEMENT...")
print("=" * 60 + "\n")

# Charger le modèle pré-entraîné
model = YOLO('yolov8n-cls.pt')

# Lancer l'entraînement
results = model.train(
    data='dataset_sdb',
    epochs=100,
    imgsz=224,
    batch=16,
    patience=15,
    device=device,
    name='sdb_classifier',
    exist_ok=True,
    pretrained=True,
    verbose=True,
    plots=True
)

print("\n" + "=" * 60)
print("✅ ENTRAÎNEMENT TERMINÉ !")
print("=" * 60)

print(f"\n📁 Résultats sauvegardés dans :")
print(f"   runs/classify/sdb_classifier/")
print(f"\n💾 Meilleur modèle :")
print(f"   runs/classify/sdb_classifier/weights/best.pt")

print("\n🚀 PROCHAINE ÉTAPE : Cell 5 (Évaluation)")
print("=" * 60)

🚀 ENTRAÎNEMENT YOLOV8 CLASSIFICATION
   Modèle : Salles de Bain (3 classes)

💻 Device : CPU
⏱️  Durée estimée : 45-60 min

📋 Configuration :
   Modèle     : YOLOv8n-cls (nano, rapide)
   Classes    : Bas / Moyen / Haut
   Epochs     : 100 (avec early stopping)
   Patience   : 15 (arrêt si pas d'amélioration)
   Image size : 224×224
   Batch size : 16

🔥 DÉMARRAGE DE L'ENTRAÎNEMENT...

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_sdb, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015,

In [ ]:
# ===== CELL 5 : ÉVALUATION =====
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

print("=" * 60)
print("📊 ÉVALUATION DU MODÈLE")
print("=" * 60)

# Charger le meilleur modèle
model = YOLO('runs/classify/sdb_classifier/weights/best.pt')

print("\n🧪 Test sur le dataset de validation...\n")

# Évaluation
metrics = model.val(data='dataset_sdb')

print("\n" + "=" * 60)
print("✅ RÉSULTATS FINAUX")
print("=" * 60)

print(f"\n📊 PERFORMANCES GLOBALES :")
print(f"   Top-1 Accuracy : {metrics.top1*100:.2f}%")
print(f"   Top-5 Accuracy : {metrics.top5*100:.2f}%")

print("\n💡 INTERPRÉTATION :")
if metrics.top1 >= 0.85:
    print("   🏆 EXCELLENT ! Modèle très performant !")
    print("   → Précision professionnelle")
    print("   → Prêt pour usage réel")
elif metrics.top1 >= 0.75:
    print("   ✅ TRÈS BON ! Modèle fiable")
    print("   → Utilisable pour Intelligence Lab")
    print("   → Bonnes prédictions")
elif metrics.top1 >= 0.65:
    print("   ✅ BON ! Modèle fonctionnel")
    print("   → Résultats corrects")
    print("   → Peut être amélioré avec plus de données")
else:
    print("   ⚠️ MOYEN")
    print("   → Ajouter plus d'images pour améliorer")

print(f"\n📁 Tous les graphiques et résultats :")
print(f"   runs/classify/sdb_classifier/")
print(f"   ├─ results.png (courbes d'entraînement)")
print(f"   ├─ confusion_matrix.png")
print(f"   └─ val_batch*.jpg (exemples)")

# Test sur quelques images
print("\n" + "=" * 60)
print("🧪 TEST SUR IMAGES EXEMPLES")
print("=" * 60 + "\n")

test_imgs = []
for classe in ['bas', 'moyen', 'haut']:
    test_path = f"dataset_sdb/test/{classe}"
    if os.path.exists(test_path):
        imgs = [f"{test_path}/{f}" for f in os.listdir(test_path) if f.endswith('.jpg')]
        if imgs:
            test_imgs.append((imgs[0], classe))

if test_imgs:
    for img_path, true_class in test_imgs[:3]:
        print(f"📸 Image : {os.path.basename(img_path)}")
        print(f"   Vraie classe : {true_class}")

        results = model(img_path, verbose=False)

        for r in results:
            probs = r.probs
            pred_class = r.names[probs.top1]
            confidence = probs.top1conf.item()

            emoji_pred = {'bas': '🔴', 'moyen': '🟡', 'haut': '🟢'}[pred_class]
            emoji_true = {'bas': '🔴', 'moyen': '🟡', 'haut': '🟢'}[true_class]

            correct = "✅" if pred_class == true_class else "❌"

            print(f"   {correct} Prédiction : {emoji_pred} {pred_class} ({confidence*100:.1f}%)")
            print()

print("=" * 60)
print("🚀 PROCHAINE ÉTAPE : Cell 6 (Interface graphique)")
print("=" * 60)

📊 ÉVALUATION DU MODÈLE

🧪 Test sur le dataset de validation...

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLOv8n-cls summary (fused): 30 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: /content/dataset_sdb/train... found 105 images in 3 classes ✅ 
val: /content/dataset_sdb/valid... found 30 images in 3 classes ✅ 
test: /content/dataset_sdb/test... found 15 images in 3 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1518.5±299.3 MB/s, size: 196.6 KB)
val: Scanning /content/dataset_sdb/valid... 30 images, 0 corrupt: 100% ━━━━━━━━━━━━ 30/30 27.4Kit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 2/2 2.1it/s 1.0s
                   all      0.733          1
Speed: 0.0ms preprocess, 12.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val

✅ RÉSULTATS FINAUX

📊 PERFORMANCES GLOBALES :
   Top-1 Accuracy : 73.33%
   Top-5 Accuracy : 100.00%

💡 INTERPRÉTATI

In [ ]:
# ===== CELL 6 : INTERFACE SANS GRADIO (UPLOAD MANUEL) =====
from google.colab import files
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

print("=" * 60)
print("📤 INTERFACE DE TEST SIMPLE")
print("=" * 60)

# Charger modèle
model = YOLO('runs/classify/sdb_classifier/weights/best.pt')
print("\n✅ Modèle chargé\n")

# Infos
infos = {
    'bas': {'emoji': '🔴', 'nom': 'BAS - Travaux nécessaires', 'cout': '10-20k€'},
    'moyen': {'emoji': '🟡', 'nom': 'MOYEN - État correct', 'cout': '3-8k€'},
    'haut': {'emoji': '🟢', 'nom': 'HAUT - Excellent état', 'cout': '0-1k€'}
}

print("📤 Upload une image de salle de bain :\n")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\n{'='*60}")
    print(f"📸 Analyse de : {filename}")
    print('='*60)

    # Charger image
    img = Image.open(filename)

    # Afficher l'image
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(filename)
    plt.show()

    # Prédiction
    results = model(img, verbose=False)
    result = results[0]
    probs = result.probs

    # Résultats
    classe = result.names[probs.top1]
    conf = probs.top1conf.item() * 100
    info = infos[classe]

    print(f"\n🎯 RÉSULTAT :\n")
    print(f"{info['emoji']} {info['nom']}")
    print(f"   Confiance : {conf:.1f}%")
    print(f"   Coût travaux : {info['cout']}")

    print(f"\n📊 PROBABILITÉS :\n")
    for i in range(len(probs.data)):
        nom = result.names[i]
        prob = probs.data[i].item() * 100
        emoji = infos[nom]['emoji']
        print(f"   {emoji} {nom.upper():<10} : {prob:5.1f}%")

    print(f"\n{'='*60}\n")

print("\n✅ Analyse terminée !")
print("\n💡 Pour tester une autre image :")
print("   → Re-exécute cette cell")

📤 INTERFACE DE TEST SIMPLE

✅ Modèle chargé

📤 Upload une image de salle de bain :



KeyboardInterrupt: 